In [21]:
#import clusim.sim as csim
#import leidenalg
#from igraph import Graph
#from clusim.clustering import Clustering, print_clustering
import pandas as pd
#from infomap import Infomap
import numpy as np
import xnetwork as xn
#import graph_tool as gt
#import graph_tool.inference as gtInference
#from graph_tool import Graph as gtGraph
from pathlib import Path 
from tqdm.auto import tqdm
#from multiprocessing import Pool
#from functools import partial
from sklearn.decomposition import PCA
from collections import defaultdict
#from functools import partial

In [22]:
data_demo=pd.read_csv('KBASE_datashare\\demo_final2024.merge_phenotype.506_v1.csv')
data_demo.head()

,Last.Name,Collection_time,Timepoint.Label,AmyPos,Barcode,Batch,RIN.Value,Sex,Edu,APOE,...,GlCtx_CL_log,LatParLobe_log,TotalHippVol,EntCtx_Thick,ICV,ICV_log,EXF_EP,LAN_EP,MEM_EP,VSP_EP
0,BR0001,BL,0,0.0,42428646,7th,8.4,0,12,E3/3,...,0.059412,0.063071,7900.9,3.575866,1531891.714,6.185228,1.347,1.086,0.683,1.569
1,BR0003,BL,0,0.0,42428655,5th,8.2,1,18,E3/3,...,0.060887,0.081167,7239.8,2.482422,1688386.805,6.227472,-0.217,0.167,-0.337,-0.146
2,BR0004,BL,0,1.0,42428635,10th,8.2,0,6,E3/4,...,0.253653,0.241970,7494.4,2.867712,1378396.803,6.139374,-0.537,-0.782,0.194,-2.050
3,BR0005,BL,0,1.0,42428621,8th,8.2,1,12,E3/4,...,0.253556,0.246080,6842.1,3.425107,1594166.455,6.202534,-0.094,0.043,-0.603,0.286
4,BR0009,BL,0,0.0,42428626,3th,2.4,0,6,E3/3,...,0.073498,0.084004,6915.4,3.859623,1337054.198,6.126149,1.039,0.709,-0.083,0.566


In [23]:
data_GlCtx_CL_log=data_demo[['Barcode','GlCtx_CL_log']]
data_GlCtx_CL_log.set_index('Barcode', inplace=True)
data_GlCtx_CL_log= data_GlCtx_CL_log.T
data_GlCtx_CL_log.head()

Barcode,42428646,42428655,42428635,42428621,42428626,42428612,42428592,42428601,42428752,42428760,...,42719122,42719148,42719156,42719178,42718998,42718988,42718975,42718947,42716172,42716168
GlCtx_CL_log,0.059412,0.060887,0.253653,0.253556,0.073498,0.076568,0.259713,0.050341,0.05873,0.101163,...,0.239124,0.180871,0.207661,0.131522,0.185684,0.049412,0.222586,0.23621,0.315614,0.03985


In [4]:
data_VSP_EP=data_demo[['Barcode','VSP_EP']]
data_VSP_EP.set_index('Barcode', inplace=True)
data_VSP_EP= data_VSP_EP.T
data_VSP_EP.head()

Barcode,42428646,42428655,42428635,42428621,42428626,42428612,42428592,42428601,42428752,42428760,...,42719122,42719148,42719156,42719178,42718998,42718988,42718975,42718947,42716172,42716168
VSP_EP,1.569,-0.146,-2.05,0.286,0.566,-2.091,-1.624,-1.717,0.224,-1.81,...,-1.299,-1.403,-0.547,-1.628,0.352,0.566,0.352,NaN,-1.997,1.314


In [24]:
data_deposition=pd.read_csv('KBASE_datashare\\residuals_cqn_cpm.13603_1140.csv')
data_deposition.set_index('Barcode', inplace=True)
data_deposition=data_deposition.T
data_deposition.head()

Barcode,42425466,42425485,42425503,42425507,42425517,42425526,42425536,42425546,42425554,42425569,...,42719148,42719156,42719178,42729014,42729035,42729099,42729103,42729190,42729209,42733238
ENSG00000000419.14,-0.025602,0.167509,0.013347,0.504928,0.025635,-0.174476,0.004188,-0.091964,0.114177,-0.287812,...,-0.029503,0.028182,-0.332341,0.054127,-0.041002,-0.134013,0.017590,0.015801,0.150996,0.023320
ENSG00000000457.14,-0.120398,-0.016713,-0.036949,0.135288,0.080433,-0.051730,-0.018096,-0.102710,0.121348,-0.091808,...,-0.173309,0.311596,-0.213821,-0.021312,-0.059251,-0.055336,0.052658,0.128468,0.072779,-0.148871
ENSG00000000460.17,0.045326,-0.159610,-0.325952,-0.074574,0.150633,-0.097574,0.082463,0.032035,0.168103,0.099374,...,0.169062,0.122287,0.053825,0.107441,-0.117710,-0.063597,-0.217800,0.088137,0.083201,0.022423
ENSG00000000938.13,0.035264,-0.176183,0.285365,0.263099,-0.082066,-0.130983,-0.046495,-0.031446,-0.109125,-0.094433,...,-0.200148,0.108318,0.113011,0.065125,0.132820,0.310166,0.098510,0.088131,0.265204,0.091941
ENSG00000000971.17,0.910862,-1.307548,0.250657,0.499940,-1.035806,1.059606,0.393797,-0.466189,-0.557025,0.469142,...,0.095053,-0.840107,1.236145,0.481200,1.027841,0.063120,-0.162505,1.234146,-0.063983,0.370594


In [25]:
def calculate_eigengene(values_df):


    beta_values_matrix = values_df.values
    pca = PCA(n_components=1)
    pca.fit(beta_values_matrix)
    eigengene_values = pca.components_
    return pd.DataFrame(eigengene_values,columns=data_deposition.columns)

In [ ]:
from scipy.stats import pearsonr
# original function
def calculateCorrelations(cg_names): 
    data_module = data_deposition.loc[data_deposition.index.isin(cg_names)]
    eigengene_df = calculate_eigengene(data_module)

    series1 = eigengene_df.iloc[0]
    series2 = data_GlCtx_CL_log[eigengene_df.columns].iloc[0]

    correlation = series1.corr(series2) # convenient with Pandas, uses pearson by default
    return {"correlation": correlation}


In [44]:
import pingouin as pg

# Alternative: use Biweight midcorrelation (the one in WGCNA)
def calculateCorrelations_Biweight_midcorrelation(cg_names): 
    data_module = data_deposition.loc[data_deposition.index.isin(cg_names)]
    eigengene_df = calculate_eigengene(data_module)

    series1 = eigengene_df.iloc[0]
    series2 = data_GlCtx_CL_log[eigengene_df.columns].iloc[0]
    result = pg.corr(series1, series2, method='bicor')
    correlation = result['r'].values[0]
    pval = result['p-val'].values[0]
    return {"correlation": correlation, "pval": pval}

In [41]:
from scipy.stats import pearsonr
import numpy as np

def calculateCorrelations(cg_names):
    data_module = data_deposition.loc[data_deposition.index.isin(cg_names)]
    eigengene_df = calculate_eigengene(data_module)

    series1 = eigengene_df.iloc[0]
    series2 = data_GlCtx_CL_log[eigengene_df.columns].iloc[0]

    # Combine into DataFrame for easier NaN/Inf dropping
    df = pd.DataFrame({'s1': series1, 's2': series2})
    df = df.replace([np.inf, -np.inf], np.nan).dropna()

    # Compute Pearson correlation and p-value
    correlation, pval = pearsonr(df['s1'], df['s2'])

    return {"correlation": correlation, "pval": pval}


In [45]:
def membership_to_clusters(membership_vector):

    clusters = defaultdict(set)

    for idx, cluster_id in enumerate(membership_vector):
        clusters[cluster_id].add(idx)
    

    cluster_list = [set(cluster) for cluster in clusters.values()]
    
    return cluster_list

In [46]:
resolutions = [0.001,0.005,0.01,0.1,1.0,0.5,5.0,10.0,20.0,50,100]
markovTimes = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1,5,10]

def processnetwork(networkFile):

    g = xn.load(networkFile)
    entries = []

    for res in resolutions:

        #leiden weighted
        propertyName = "Leiden_weighted_%f"%res

        membership_list=g.vs[propertyName]

        clusters = membership_to_clusters(membership_list)

        for c in clusters:
            cg_names=[g.vs[idx]['Label'] for idx in c]
            entries.append({"cluster": cg_names,
                            "cluster size":len(cg_names),
                            "resolution":res,
                            "network":networkFile.stem,
                            "method":'leiden_weighted',
                            })

            entries[-1].update(calculateCorrelations_Biweight_midcorrelation(cg_names))
        
        
        
        #leiden unweighted
        propertyName = "Leiden_weighted_%f"%res

        membership_list=g.vs[propertyName]

        clusters = membership_to_clusters(membership_list)

        for c in clusters:
            
            cg_names=[g.vs[idx]['Label'] for idx in c]
            entries.append({"cluster": cg_names,
                            "cluster size":len(cg_names),
                            "resolution":res,
                            "network":networkFile.stem,
                            "method":'leiden_unweighted',
                            })

            entries[-1].update(calculateCorrelations_Biweight_midcorrelation(cg_names))
    del g
    return entries

In [47]:


if __name__ == "__main__":
    networksWithCommunitiesPath = Path("KNN_NetworksWithCommunities")
    networkFiles = list(networksWithCommunitiesPath.glob("*.xnet"))
    print(networkFiles)
    allEntries = []
    for networkFile in networkFiles:
        entries = processnetwork(networkFile)
        allEntries += entries
    
    df = pd.DataFrame(allEntries)
    df.to_csv("Correlation_results\\module_correlations_leiden_GlCtx_CL_log_p_value_BiCor.csv", index=False)



[WindowsPath('KNN_NetworksWithCommunities/knn_10.xnet'), WindowsPath('KNN_NetworksWithCommunities/knn_10_pos.xnet'), WindowsPath('KNN_NetworksWithCommunities/knn_11.xnet'), WindowsPath('KNN_NetworksWithCommunities/knn_11_pos.xnet'), WindowsPath('KNN_NetworksWithCommunities/knn_12.xnet'), WindowsPath('KNN_NetworksWithCommunities/knn_12_pos.xnet'), WindowsPath('KNN_NetworksWithCommunities/knn_13.xnet'), WindowsPath('KNN_NetworksWithCommunities/knn_13_pos.xnet'), WindowsPath('KNN_NetworksWithCommunities/knn_14.xnet'), WindowsPath('KNN_NetworksWithCommunities/knn_14_pos.xnet'), WindowsPath('KNN_NetworksWithCommunities/knn_15.xnet'), WindowsPath('KNN_NetworksWithCommunities/knn_15_pos.xnet'), WindowsPath('KNN_NetworksWithCommunities/knn_4.xnet'), WindowsPath('KNN_NetworksWithCommunities/knn_4_pos.xnet'), WindowsPath('KNN_NetworksWithCommunities/knn_5.xnet'), WindowsPath('KNN_NetworksWithCommunities/knn_5_pos.xnet'), WindowsPath('KNN_NetworksWithCommunities/knn_6.xnet'), WindowsPath('KNN_Net

c:\Users\merit\anaconda3\Lib\site-packages\sklearn\decomposition\_pca.py:541: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\merit\anaconda3\Lib\site-packages\sklearn\decomposition\_pca.py:541: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\merit\anaconda3\Lib\site-packages\sklearn\decomposition\_pca.py:541: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\merit\anaconda3\Lib\site-packages\sklearn\decomposition\_pca.py:541: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\merit\anaconda3\Lib\site-packages\sklearn\decomposition\_pca.py:541: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
c:\Users\merit\anaconda3\Lib\site-packages\sklearn\decomposition\_pca.py:541: RuntimeWarning: invalid val